# 数据库交互式分析模板

本笔记本提供了与 alphahome 数据库进行交互式分析的完整示例，包括：
- 数据库连接和配置
- 数据查询和探索
- 数据质量检查
- 基础统计分析
- 数据可视化

## 环境设置

In [ ]:
# 导入必要的库
import sys
import os
import asyncio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

# 添加项目根目录到路径
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../../..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"项目根目录: {project_root}")
print(f"当前工作目录: {os.getcwd()}")

## 数据库连接设置

In [ ]:
# 导入 alphahome 组件
from alphahome.common.config_manager import ConfigManager
from alphahome.common.db_manager import DBManager

# 初始化配置管理器
config_manager = ConfigManager()
config = config_manager.load_config()

print("配置加载成功")
print(f"数据库URL: {config.get('database', {}).get('url', 'Not configured')}")

In [ ]:
# 创建数据库连接
async def create_db_connection():
    """创建异步数据库连接"""
    db_manager = DBManager()
    await db_manager.initialize()
    return db_manager

# 在 Jupyter 中运行异步代码
try:
    # 检查是否已有事件循环
    loop = asyncio.get_event_loop()
    if loop.is_running():
        # 在已运行的事件循环中创建任务
        import nest_asyncio
        nest_asyncio.apply()
        db = await create_db_connection()
    else:
        db = await create_db_connection()
except RuntimeError:
    # 创建新的事件循环
    db = asyncio.run(create_db_connection())

print("数据库连接创建成功")

## 数据库结构探索

In [ ]:
# 查看所有表
async def get_all_tables():
    """获取数据库中所有表的信息"""
    query = """
    SELECT 
        schemaname,
        tablename,
        tableowner
    FROM pg_tables 
    WHERE schemaname NOT IN ('information_schema', 'pg_catalog')
    ORDER BY schemaname, tablename;
    """
    return await db.fetch_all(query)

tables = await get_all_tables()
tables_df = pd.DataFrame(tables)

print(f"数据库中共有 {len(tables_df)} 个表:")
print(tables_df.to_string(index=False))

In [ ]:
# 查看表的详细信息
async def get_table_info(table_name, schema='public'):
    """获取指定表的列信息"""
    query = """
    SELECT 
        column_name,
        data_type,
        is_nullable,
        column_default
    FROM information_schema.columns 
    WHERE table_schema = $1 AND table_name = $2
    ORDER BY ordinal_position;
    """
    return await db.fetch_all(query, schema, table_name)

# 示例：查看股票基础信息表结构（如果存在）
sample_tables = ['tushare_stock_basic', 'tushare_stock_daily', 'stock_basic']

for table_name in sample_tables:
    try:
        table_exists = await db.table_exists(table_name)
        if table_exists:
            print(f"\n=== {table_name} 表结构 ===")
            columns = await get_table_info(table_name)
            columns_df = pd.DataFrame(columns)
            print(columns_df.to_string(index=False))
            break
    except Exception as e:
        print(f"检查表 {table_name} 时出错: {e}")
        continue
else:
    print("未找到常见的股票数据表，请检查数据库中的表名")

## 数据查询和探索

In [ ]:
# 查询示例数据
async def query_sample_data(table_name, limit=10):
    """查询表的示例数据"""
    query = f"SELECT * FROM {table_name} LIMIT $1"
    return await db.fetch_all(query, limit)

# 获取数据表行数统计
async def get_table_stats():
    """获取各表的行数统计"""
    stats = []
    for table in tables:
        table_name = table['tablename']
        schema_name = table['schemaname']
        full_name = f"{schema_name}.{table_name}" if schema_name != 'public' else table_name
        
        try:
            count_query = f"SELECT COUNT(*) as count FROM {full_name}"
            result = await db.fetch_one(count_query)
            stats.append({
                'schema': schema_name,
                'table': table_name,
                'row_count': result['count']
            })
        except Exception as e:
            print(f"查询表 {full_name} 行数时出错: {e}")
            stats.append({
                'schema': schema_name,
                'table': table_name,
                'row_count': 'Error'
            })
    
    return stats

# 执行统计查询
table_stats = await get_table_stats()
stats_df = pd.DataFrame(table_stats)

print("数据表行数统计:")
print(stats_df.to_string(index=False))

In [ ]:
# 选择一个有数据的表进行详细分析
# 找到行数最多的表
numeric_stats = stats_df[stats_df['row_count'] != 'Error'].copy()
if not numeric_stats.empty:
    numeric_stats['row_count'] = pd.to_numeric(numeric_stats['row_count'])
    largest_table = numeric_stats.loc[numeric_stats['row_count'].idxmax()]
    
    target_table = largest_table['table']
    target_schema = largest_table['schema']
    full_table_name = f"{target_schema}.{target_table}" if target_schema != 'public' else target_table
    
    print(f"选择表 '{full_table_name}' 进行详细分析 (行数: {largest_table['row_count']:,})")
    
    # 查看示例数据
    sample_data = await query_sample_data(full_table_name, 5)
    sample_df = pd.DataFrame(sample_data)
    
    print(f"\n{target_table} 示例数据:")
    print(sample_df.head())
    
    # 获取表结构
    table_columns = await get_table_info(target_table, target_schema)
    columns_df = pd.DataFrame(table_columns)
    
    print(f"\n{target_table} 表结构:")
    print(columns_df.to_string(index=False))
    
else:
    print("未找到可分析的数据表")
    target_table = None

## 数据质量检查

In [ ]:
if target_table:
    # 数据质量检查
    async def data_quality_check(table_name):
        """执行数据质量检查"""
        quality_report = {}
        
        # 总行数
        count_result = await db.fetch_one(f"SELECT COUNT(*) as total_rows FROM {table_name}")
        quality_report['total_rows'] = count_result['total_rows']
        
        # 获取所有列名
        columns_query = """
        SELECT column_name, data_type 
        FROM information_schema.columns 
        WHERE table_name = $1 AND table_schema = 'public'
        ORDER BY ordinal_position
        """
        columns = await db.fetch_all(columns_query, table_name.split('.')[-1])
        
        # 检查每列的空值情况
        null_checks = []
        for col in columns:
            col_name = col['column_name']
            null_query = f"""
            SELECT 
                COUNT(*) as total_count,
                COUNT({col_name}) as non_null_count,
                COUNT(*) - COUNT({col_name}) as null_count
            FROM {table_name}
            """
            try:
                result = await db.fetch_one(null_query)
                null_checks.append({
                    'column': col_name,
                    'data_type': col['data_type'],
                    'total_count': result['total_count'],
                    'non_null_count': result['non_null_count'],
                    'null_count': result['null_count'],
                    'null_percentage': (result['null_count'] / result['total_count'] * 100) if result['total_count'] > 0 else 0
                })
            except Exception as e:
                print(f"检查列 {col_name} 时出错: {e}")
        
        quality_report['column_quality'] = null_checks
        return quality_report
    
    # 执行数据质量检查
    quality_report = await data_quality_check(full_table_name)
    
    print(f"数据质量报告 - {target_table}")
    print(f"总行数: {quality_report['total_rows']:,}")
    print("\n列质量统计:")
    
    quality_df = pd.DataFrame(quality_report['column_quality'])
    print(quality_df[['column', 'data_type', 'null_count', 'null_percentage']].to_string(index=False))

## 自定义查询区域

在这里你可以执行自定义的SQL查询来探索数据

In [ ]:
# 自定义查询示例
async def execute_custom_query(query, params=None):
    """执行自定义查询"""
    try:
        if params:
            result = await db.fetch_all(query, *params)
        else:
            result = await db.fetch_all(query)
        return pd.DataFrame(result)
    except Exception as e:
        print(f"查询执行出错: {e}")
        return None

# 你的自定义查询区域
# 在这里编写和执行你自己的SQL查询

# 示例：
# my_query = "SELECT COUNT(*) FROM your_table WHERE condition"
# result = await execute_custom_query(my_query)
# print(result)

print("自定义查询区域 - 请在上方编写你的查询")

## 数据导出功能

In [ ]:
# 数据导出功能
async def export_data_to_csv(query, filename, params=None):
    """将查询结果导出为CSV文件"""
    try:
        df = await execute_custom_query(query, params)
        if df is not None and not df.empty:
            # 确保导出目录存在
            export_dir = os.path.join(os.getcwd(), 'exports')
            os.makedirs(export_dir, exist_ok=True)
            
            filepath = os.path.join(export_dir, filename)
            df.to_csv(filepath, index=False, encoding='utf-8-sig')
            print(f"数据已导出到: {filepath}")
            print(f"导出行数: {len(df)}")
            return filepath
        else:
            print("没有数据可导出")
            return None
    except Exception as e:
        print(f"导出数据时出错: {e}")
        return None

# 示例导出
if target_table:
    export_query = f"SELECT * FROM {full_table_name} LIMIT 1000"
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    export_filename = f"{target_table}_sample_{timestamp}.csv"
    
    print(f"导出 {target_table} 的样本数据...")
    exported_file = await export_data_to_csv(export_query, export_filename)
    
    if exported_file:
        print(f"导出完成: {exported_file}")

## 清理和关闭连接

In [ ]:
# 关闭数据库连接
try:
    await db.close()
    print("数据库连接已关闭")
except Exception as e:
    print(f"关闭连接时出错: {e}")

print("\n=== 分析完成 ===")
print("本笔记本提供了完整的数据库交互式分析模板")
print("你可以根据需要修改查询和分析逻辑")

## 总结

本笔记本演示了如何使用 alphahome 框架进行数据库交互式分析，包括：

1. **环境设置**: 导入必要的库和配置项目路径
2. **数据库连接**: 使用 ConfigManager 和 DBManager 建立连接
3. **结构探索**: 查看数据库中的表和列结构
4. **数据查询**: 执行各种查询获取数据
5. **质量检查**: 检查数据完整性和质量
6. **自定义查询**: 提供灵活的查询接口
7. **数据导出**: 将分析结果导出为CSV文件

### 使用建议

- 根据你的具体数据结构调整查询语句
- 在处理大量数据时注意使用适当的LIMIT子句
- 定期保存重要的分析结果
- 可以将常用的分析逻辑封装成函数以便重复使用

### 扩展功能

你可以基于这个模板添加更多功能：
- 机器学习模型训练
- 更复杂的统计分析
- 交互式图表
- 自动化报告生成
- 数据监控和告警